In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..","..")))

In [2]:
from langchain_experimental.agents import create_pandas_dataframe_agent as create_default_agent

In [ ]:

PREFIX = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:"""

SUFFIX_NO_DF = """
Begin!
Question: {input}
{agent_scratchpad}"""

SUFFIX_WITH_DF = """
Here is the documentation for the dataframe:
{df_doc}

Begin!
Question: {input}
{agent_scratchpad}"""

import warnings
from typing import Any, Dict, List, Literal, Optional, Sequence, Union, cast
import pandas as pd

from langchain.agents import (
    AgentType,
    create_react_agent,
)
from langchain.agents.agent import (
    AgentExecutor,
    BaseMultiActionAgent,
    BaseSingleActionAgent,
    RunnableAgent,
)
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS

from langchain_core.callbacks import BaseCallbackManager
from langchain_core.language_models import LanguageModelLike
from langchain_core.prompts import (
    BasePromptTemplate,
    PromptTemplate,
)
from langchain_core.tools import BaseTool
from langchain_core.utils.interactive_env import is_interactive_env

from langchain_experimental.tools.python.tool import PythonAstREPLTool

from tools.rag_tool import RAGTool


def _get_single_prompt(
    df: Any,
    *,
    prefix: Optional[str] = None,
    suffix: Optional[str] = None,
    include_df_in_prompt: Optional[bool] = True,
    number_of_head_rows: int = 5,
) -> BasePromptTemplate:
    if suffix is not None:
        suffix_to_use = suffix
    elif include_df_in_prompt:
        suffix_to_use = SUFFIX_WITH_DF
    else:
        suffix_to_use = SUFFIX_NO_DF
    prefix = prefix if prefix is not None else PREFIX

    template = "\n\n".join([prefix, "{tools}", FORMAT_INSTRUCTIONS, suffix_to_use])
    prompt = PromptTemplate.from_template(template)

    partial_prompt = prompt.partial()

    return partial_prompt


def _get_prompt(df: Any, **kwargs: Any) -> BasePromptTemplate:
    return _get_single_prompt(df, **kwargs)


def create_pandas_dataframe_agent(
    llm: LanguageModelLike,
    df: Any,
    agent_type: Union[
        AgentType, Literal["openai-tools", "tool-calling"]
    ] = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callback_manager: Optional[BaseCallbackManager] = None,
    prefix: Optional[str] = None,
    suffix: Optional[str] = None,
    input_variables: Optional[List[str]] = None,
    verbose: bool = False,
    return_intermediate_steps: bool = False,
    max_iterations: Optional[int] = 15,
    max_execution_time: Optional[float] = None,
    early_stopping_method: str = "force",
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    include_df_in_prompt: Optional[bool] = True,
    number_of_head_rows: int = 5,
    extra_tools: Sequence[BaseTool] = (),
    engine: Literal["pandas", "modin"] = "pandas",
    allow_dangerous_code: bool = False,
    **kwargs: Any,
) -> AgentExecutor:
    if not allow_dangerous_code:
        raise ValueError(
            "This agent relies on access to a python repl tool which can execute "
            "arbitrary code. This can be dangerous and requires a specially sandboxed "
            "environment to be safely used. Please read the security notice in the "
            "doc-string of this function. You must opt-in to use this functionality "
            "by setting allow_dangerous_code=True."
            "For general security guidelines, please see: "
            "https://python.langchain.com/docs/security/"
        )
    if is_interactive_env():
        pd.set_option("display.max_columns", None)

    if not isinstance(df, pd.DataFrame):
        raise ValueError(f"Expected pandas DataFrame, got {type(df)}")

    if input_variables:
        kwargs = kwargs or {}
        kwargs["input_variables"] = input_variables
    if kwargs:
        warnings.warn(
            f"Received additional kwargs {kwargs} which are no longer supported."
        )

    df_locals = {}

    df_locals["df"] = df
    tools = [PythonAstREPLTool(locals=df_locals),RAGTool()]

    if include_df_in_prompt is not None and suffix is not None:
        raise ValueError("If suffix is specified, include_df_in_prompt should not be.")
    prompt = _get_prompt(
        df,
        prefix=prefix,
        suffix=suffix,
        include_df_in_prompt=include_df_in_prompt,
        number_of_head_rows=number_of_head_rows,
    )

    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent] = RunnableAgent(
        runnable=create_react_agent(llm, tools, prompt),  # type: ignore
        input_keys_arg=["input"],
        return_keys_arg=["output"],
    )
    return AgentExecutor(
        agent=agent,
        tools=tools,
        callback_manager=callback_manager,
        verbose=verbose,
        return_intermediate_steps=return_intermediate_steps,
        max_iterations=max_iterations,
        max_execution_time=max_execution_time,
        early_stopping_method=early_stopping_method,
        **(agent_executor_kwargs or {}),
    )

In [6]:
from langchain.agents.agent_types import AgentType
import pandas as pd
from langchain_openai.chat_models.base import BaseChatOpenAI
import os

In [7]:
llm = BaseChatOpenAI(
    model="deepseek-chat",
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com",
    max_tokens=8192,
    temperature=0.0,
)

In [8]:
class Test:
    def __init__(self):
        self.df=None
    def get_df(self):
        return self.df
    def set_df(self, df):
        self.df = df

In [9]:
class TestDataLoadingTool:
    def __init__(self):
        self.df = pd.read_csv("../../final_data/data/2024-12-01.csv", sep=",")

    def get_dataframe(self):
        return self.df

In [10]:
test = TestDataLoadingTool()

In [ ]:
test.get_dataframe()["VendorName"].value_counts().idxmax()

In [12]:
default_agent = create_default_agent(
    llm=llm,
    df=test.get_dataframe(),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    verbose=True,
)

custom_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=test.get_dataframe(),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    verbose=True,
    include_df_in_prompt=False
)

Set pandas display.max_columns to None for interactive env
Checking type of dataframe: <class 'pandas.core.frame.DataFrame'>
Added df to df_locals


/Users/haithem/Documents/PFE/Proof_of_Concept/src/tools/rag_tool.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.data_embeddings = HuggingFaceEmbeddings(model_name=self.model_name)


Called _get_single_prompt
Single prompt template:

You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


Begin!
Question: {input}
{agent_scratchpad}
Partial prompt input variables before filling: ['agent_scratchpad', 'input', 'tool_names', 'tools']
Prompt for ZERO_SHOT_REACT_DESCRIPTION: input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template='\nYou are working with a pandas dataframe in Python. The 

In [ ]:
# from langchain.callbacks.base import BaseCallbackHandler


# class LoggingHandler(BaseCallbackHandler):
#     def __init__(self):
#         self.logs = []

#     def on_agent_action(self, action, **kwargs):
#         self.logs.append(f"[Thought+Action]\n{action.log}\n")

#     def on_tool_end(self, output, **kwargs):
#         self.logs.append(f"[Tool Output]\n{output}\n")

#     def on_agent_finish(self, finish, **kwargs):
#         self.logs.append(f"[Final Answer]\n{finish.get('output', '')}\n")
    

In [14]:
import time


def benchmark_agent(prompts, filename, agent, K=10):
    logs_df = []

    for prompt in prompts:
        for i in range(K):
            handler = LoggingHandler()
            start_time = time.time()
            try:
                response = agent.run(prompt, callbacks=[handler])
            except Exception as e:
                response = f"Error: {str(e)}"
            duration = time.time() - start_time

            logs_df.append(
                {
                    "prompt": prompt,
                    "run_id": i + 1,
                    "final_answer": response,
                    "duration_sec": round(duration, 3),
                    "reasoning_trace": "\n".join(handler.logs),
                }
            )

    df = pd.DataFrame(logs_df)
    df.to_csv(filename + ".csv", index=False)

In [13]:
questionsAnalyse = [
    "What is the average loading delay per supplier?",
    "Which docks are most frequently blocked and what is the impact on loading schedules?",
    "What is the distribution of order statuses over the last 30 days?",
    "Which suppliers handle the largest volume of transport orders?",
    "How many orders are assigned to the El Kseur docks compared to other regions?",
    "What percentage of docks handle palletized goods compared to big bags?",
    "What are the 5 most transported products by quantity?",
    "How many orders were handled by third-party agencies compared to internal transport?",
    "Which customer generated the most transport orders this quarter?",
    "What are the most frequent differences between the planned loading date and the actual date?",
]

In [14]:
prompts = [
    "Which supplier has the most orders?",
    "Give me a summary of the order statuses.",
    "Which type of product has the most orders?",
    "What is the most frequent order destination?",
]

In [15]:
challenging_prompt = """
Hey there!  

I need your help analyzing our transport and dock operations. We’ve been having some delays and inefficiencies, and I’d like you to dig into the data to find out what’s going on. Here’s what I need:  

1. Dock Bottlenecks
   - Which docks are blocked most often? Are certain types of docks (like those handling big bags or pallets) more likely to be blocked?  
   - Do blocked docks cause delays in order completion?

2. Vender Performance Check  
   - Which vendors complete orders fastest? Show me the top and bottom 3 vendors.  
   - Do vendors using external agencies perform worse than those handling transport themselves?  

3. Mismatch Alerts  
   - Are we sending the wrong types of goods to docks?  How often does this happen, and does it cause issues?  

4. El Kseur Delays  
   - Orders going through El Kseur docks seem slower. Is this because of distance, vendor assignments, or something else?  

"""

In [ ]:
custom_agent.run(challenging_prompt)

In [ ]:
default_agent.run(challenging_prompt)

In [ ]:
benchmark_agent(prompts, "custom_rag_agent", custom_agent, 2)
benchmark_agent(prompts, "default_agent", default_agent, 2)

In [19]:
custom_df = pd.read_csv("custom_rag_agent.csv")
default_df = pd.read_csv("default_agent.csv")

In [20]:
custom_df["duration_sec"].sum(),default_df ["duration_sec"].sum()

(np.float64(260.05899999999997), np.float64(130.00900000000001))

In [ ]:
custom_df

In [ ]:
default_df